In [3]:
import pandas as pd
import numpy as np
# from operator import itemgetter
from sentence_transformers import SentenceTransformer

In [4]:
df = pd.read_csv('results/data.gov.lt.csv')[:200]
df['id'] = df.index
corpus = list(df['Name of the dataset'].values)

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(corpus)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:10]
)


array([[0.9042527 , 0.8819117 , 0.4384631 , 0.412204  , 0.30720535,
        0.40834224, 0.4691224 , 0.4781106 , 0.49695882]], dtype=float32)

In [7]:
def get_top(X,rank):
    cosines = {}
    cosines_n = {}
    for x in range(X.shape[0]):
        cossims = [cosine_similarity(x, X)]
        # print('I', x)
        d = {}
        dn = {}
        for y in range(X.shape[0]):
            if x != y:
                cossim = cosine_similarity(X[x],X[y])
                d[corpus[y]] = cossim
                dn[y] = cossim
        # d = {corpus[y]:cosine_similarity(X[x],X[y])[0][0] for y in range (X.shape[0]) if x != y}
        cosdict = dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:rank])
        cos_n_dict = dict(sorted(dn.items(), key = itemgetter(1), reverse = True)[:rank])
        cosines[x] =  '\n'.join([str(k) + ':'+ str(v) for k,v in cosdict.items()])
        cosines_n[x] =  ';'.join([str(k) + ':'+ str(v) for k,v in cos_n_dict.items()])
        print(corpus[x], cosines[x])
    return cosines, cosines_n

In [8]:
from operator import itemgetter
sdict = {}
for x in range(sentence_embeddings.shape[0]):
        cossims = cosine_similarity([sentence_embeddings[x]],sentence_embeddings)[0]
        d = dict(zip(corpus,cossims))
        d.pop(corpus[x])
        cosdict = dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:5])
        # cossims = [cosine_similarity(sentence_embeddings[x], sentence_embeddings)]
        print(corpus[x])
        print(cosdict)
        sdict[x] = '\n'.join([str(k) + ':'+ str(v) for k,v in cosdict.items()])
print(sdict)
print(len(sdict))


List of companies licensed to produce beer
{'List of companies licensed to produce alcoholic products': 0.9042527, 'List of companies holding licenses for the production of alcoholic products': 0.8819117, 'Domestic sales of some dairy products produced in Lithuanian dairy processing companies': 0.61178863, 'Wholesale and retail trade of alcoholic and tobacco products of economic entities Annual planned inspection plans': 0.561358, 'LICENSES issued to economic entities by state institutions': 0.5468349}
List of companies licensed to produce alcoholic products
{'List of companies holding licenses for the production of alcoholic products': 0.9612642, 'List of companies licensed to produce beer': 0.9042527, 'Wholesale and retail trade of alcoholic and tobacco products of economic entities Annual planned inspection plans': 0.64772266, 'LICENSES issued to economic entities by state institutions': 0.5749256, 'Classifications of JAR legal forms and statuses': 0.5706594}
List of companies holdi

In [9]:
df['similar'] = df['id'].map(sdict)


In [10]:
df.to_csv('results/data.gov.lt-simbert.csv')